In [4]:
import pandas as pd
import numpy as np
import datetime as datetime
from sklearn import linear_model, datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from pandas import Series

df = pd.read_csv(r"E:/工作/河北项目/国网试题汇编/20191208赵佩-试题书/分类_家庭用电安全隐患分析-试题/分类_家庭用电安全隐患分析_试题/Household Electric Power Consumption.csv", low_memory=False)
df = df.replace("?", 0)
df['Global_active_power'] = pd.to_numeric(df['Global_active_power'])
df['Global_reactive_power'] = pd.to_numeric(df['Global_reactive_power'])
df['Voltage'] = pd.to_numeric(df['Voltage'])
df['Global_intensity'] = pd.to_numeric(df['Global_intensity'])
df['Sub_metering_1'] = pd.to_numeric(df['Sub_metering_1'])
df['Sub_metering_2'] = df['Sub_metering_2'].astype(np.float)

df2 = pd.read_csv(r"E:/工作/河北项目/国网试题汇编/20191208赵佩-试题书/分类_家庭用电安全隐患分析-试题/分类_家庭用电安全隐患分析_试题/train_label.csv", low_memory=False)
df["Date"] = df["Date"].map(lambda x: datetime.datetime.strptime(x, '%d/%m/%Y').date())
df2["Date"] = df2["Date"].map(lambda x: datetime.datetime.strptime(x, '%Y/%m/%d').date())
df2["label"] = "Y"

df3 = df.merge(df2, how="left",right_on=["Date","Time"], left_on=["Date","Time"])
df3 = df3.fillna("N")

train = df3[df3["Date"] < datetime.date(2008,7,1)]
test = df3[df3["Date"] >= datetime.date(2008,7,1)]
train = train.drop(['Date', 'Time'], axis = 1)
# test = test.drop(['Date', 'Time'], axis = 1)
x_train = Series(train.values[:, :-1])
y_train = Series(train['label'].values[:])
x_test = test.values[:, :-1]
#y_test = train['label']

from sklearn import svm
clf = tree.DecisionTreeClassifier()
clf.fit(x_train, y_train)
test['label'] = clf.predict(test[["Global_active_power", "Global_reactive_power", "Voltage", "Global_intensity", "Sub_metering_1","Sub_metering_2","Sub_metering_3"]])
# pd.D
# print(test.head())
test["Date"] = test["Date"].map(lambda t : t.strftime("%Y/%m/%d"))
test = test[test["label"] == "Y"]
test[["Date","Time"]].to_csv('E:/工作/河北项目/国网试题汇编/20191208赵佩-试题书/分类_家庭用电安全隐患分析-试题/分类_家庭用电安全隐患分析_试题/1-4.csv', index=False)


svm_model = svm.libsvm.fit(x_train, y_train)
test_result = svm_model.predict_proba(x_test["Sub_metering_2"])
print(test_result)





Exception: Data must be 1-dimensional